In [1]:
#This notebook attempts to translate the /util/accelwattch/accelwattch_hw_profiler/gen_hw_perf_csv.py
#That is going from hardware counters to hw_perf.csv

import pandas as pd
import numpy as np

import os
import sys
import re
import csv
from optparse import OptionParser

<h1> Finding matching kernel names</h1>

In [3]:
# kernel_name_map = {
#     # nsight name : accel-sim name
#     "findRangeK" : "findRangeK",
#     "findK": "findK",
#     "bpnn_layerforward_CUDAfloatfloatfloatfloatintint" : "_Z22bpnn_layerforward_CUDAPfS_S_S_ii",
#     "bpnn_adjust_weights_cudafloatintfloatintfloatfloat" : "_Z24bpnn_adjust_weights_cudaPfiS_iS_S_",
#     "binomialOptionsKernel" : "_Z21binomialOptionsKernelv",
#     "CUDAkernel1DCTfloatintintint" : "_Z14CUDAkernel1DCTPfiiiy",
#     "CUDAkernel2DCTfloatfloatint": "_Z14CUDAkernel2DCTPfS_i", 
#     "fwtBatch2Kernelfloatfloatint" : "_Z15fwtBatch2KernelPfS_i",
#     "fwtBatch1Kernelfloatfloatint" : "_Z15fwtBatch1KernelPfS_i",
#     "histogram64Kernelunsignedintuint4unsignedint" : "_Z17histogram64KernelPjP5uint4j",
#     "calculate_tempintfloatfloatfloatintintintintfloatfloatfloatfloatfloatfloat" : "_Z14calculate_tempiPfS_S_iiiiffffff",
#     "kmeansPointfloatintintintintfloatfloatint" : "_Z11kmeansPointPfiiiPiS_S_S0_",
#     "voidmergeElementaryIntervalsKernel1uunsignedintunsignedintunsignedintunsignedintunsignedintunsignedintunsignedintunsignedint" : "_Z30mergeElementaryIntervalsKernelILj1EEvPjS0_S0_S0_S0_S0_jj",
#     "voidmergeSortSharedKernel1uunsignedintunsignedintunsignedintunsignedintunsignedint" : "_Z21mergeSortSharedKernelILj1EEvPjS0_S0_S0_j",
#     "ComputeQ_GPUintintfloatfloatfloatfloatfloat" : "_Z12ComputeQ_GPUiiPfS_S_S_S_",
#     "mb_sad_calcunsignedshortunsignedshortintint" : "_Z11mb_sad_calcPtS_ii",
#     "mysgemmNTfloatconstintfloatconstintfloatintintfloatfloat" : "_Z9mysgemmNTPKfiS0_iPfiiff",
#     "quasirandomGeneratorKernelfloatunsignedintunsignedint" : "_Z26quasirandomGeneratorKernelPfjj",
#     "inverseCNDKernelfloatunsignedintunsignedint" : "_Z16inverseCNDKernelPfPjj",
#     "sobolGPU_kernelunsignedintunsignedintunsignedintfloat" : "_Z15sobolGPU_kerneljjPjPf",
#     "sradfloatintintlongintintintintfloatfloatfloatfloatfloatfloatfloat" : "_Z4sradfiilPiS_S_S_PfS0_S0_S0_fS0_S0_",
#     "_ZN7cutlass4gemm16gemm_kernel_nolbINS0_12GemmMainloopINS0_10GemmTraitsINS0_14WmmaGemmConfigILNS_12MatrixLayout4KindE1ELS6_1ENS_5ShapeILi64ELi128ELi128ELi1EEE6__halfS9_ffNS7_ILi64ELi32ELi64ELi1EEENS7_ILi16ELi16ELi16ELi1EEELi8ELi8ELi8ELi8ELi4ELi4ELi4EEENS0_16GlobalLoadStreamILNS_11GemmOperand4KindE0ENS0_20GemmGlobalIteratorAbINS0_20GemmGlobalTileTraitsILSF_0ELS6_1EKS9_NS7_ILi1ELi64ELi128ELi1EEENS7_ILi1ELi8ELi32ELi1EEELi8EEEiEENS_17TileStoreIteratorINS0_27GemmSharedStoreTileAbTraitsIS9_NS7_ILi1ELi64ELi136ELi1EEENS7_ILi1ELi16ELi16ELi1EEELi8EEES9_LNS_15IteratorAdvance4KindE1ELNS_11MemorySpace4KindE1EiS9_LNS_19FragmentElementType4KindE0ENS7_ILi0ELi0ELi0ELi0EEEEENS_4CopyINS_8FragmentIS9_Li32ELm16EEEEEEENSD_ILSF_1ENSG_INSH_ILSF_1ELS6_1ESI_NS7_ILi1ELi128ELi64ELi1EEENS7_ILi1ELi4ELi64ELi1EEELi8EEEiEENSN_INSO_IS9_NS7_ILi1ELi128ELi72ELi1EEENS7_ILi1ELi32ELi8ELi1EEELi8EEES9_LST_1ELSV_1EiS9_LSX_0ESY_EES13_EENS0_16SharedLoadStreamINS_16TileLoadIteratorINS0_29WmmaGemmSharedLoadTileATraitsILS6_1ES9_SP_NS7_ILi1ELi4ELi2ELi1EEELi16ENS7_ILi1ELi1ELi4ELi1EEENS7_ILi2176ELi0ELi32ELi0EEESB_EES9_LST_1ELSV_1EiNS_10WmmaMatrixILSF_0ELS6_1ES9_SB_EELSX_1ESY_EENS10_INS11_IS1M_Li4ELm16EEEEEEENS1E_INS1F_INS0_29WmmaGemmSharedLoadTileBTraitsILS6_1ES9_S19_S1H_Li1152ENS7_ILi1ELi1ELi2ELi1EEENS7_ILi16ELi0ELi4608ELi1EEESB_EES9_LST_1ELSV_1EiNS1L_ILSF_1ELS6_1ES9_SB_EELSX_1ESY_EENS10_INS11_IS1V_Li2ELm16EEEEEEENS0_12GemmEpilogueINS0_28SimplifiedGemmEpilogueTraitsISC_NS0_13LinearScalingIfNS0_19FragmentMultiplyAddIffLb1EEEEEiNS0_28WmmaGemmEpilogueTraitsHelperISC_fS25_iEEEEEENS0_20IdentityBlockSwizzleEiNS0_17ClearAccumulatorsIfLi1EEEEEEEEEvNT_6ParamsE" : "_ZN7cutlass4gemm16gemm_kernel_nolbINS0_12GemmMainloopINS0_10GemmTraitsINS0_14WmmaGemmConfigILNS_12MatrixLayout4KindE1ELS6_1ENS_5ShapeILi64ELi128ELi128ELi1EEE6__halfS9_ffNS7_ILi64ELi32ELi64ELi1EEENS7_ILi16ELi16ELi16ELi1EEELi8ELi8ELi8ELi8ELi4ELi4ELi4EEENS0_16GlobalLoadStreamILNS_11GemmOperand4KindE0ENS0_20GemmGlobalIteratorAbINS0_20GemmGlobalTileTraitsILSF_0ELS6_1EKS9_NS7_ILi1ELi64ELi128ELi1EEENS7_ILi1ELi8ELi32ELi1EEELi8EEEiEENS_17TileStoreIteratorINS0_27GemmSharedStoreTileAbTraitsIS9_NS7_ILi1ELi64ELi136ELi1EEENS7_ILi1ELi16ELi16ELi1EEELi8EEES9_LNS_15IteratorAdvance4KindE1ELNS_11MemorySpace4KindE1EiS9_LNS_19FragmentElementType4KindE0ENS7_ILi0ELi0ELi0ELi0EEEEENS_4CopyINS_8FragmentIS9_Li32ELm16EEEEEEENSD_ILSF_1ENSG_INSH_ILSF_1ELS6_1ESI_NS7_ILi1ELi128ELi64ELi1EEENS7_ILi1ELi4ELi64ELi1EEELi8EEEiEENSN_INSO_IS9_NS7_ILi1ELi128ELi72ELi1EEENS7_ILi1ELi32ELi8ELi1EEELi8EEES9_LST_1ELSV_1EiS9_LSX_0ESY_EES13_EENS0_16SharedLoadStreamINS_16TileLoadIteratorINS0_29WmmaGemmSharedLoadTileATraitsILS6_1ES9_SP_NS7_ILi1ELi4ELi2ELi1EEELi16ENS7_ILi1ELi1ELi4ELi1EEENS7_ILi2176ELi0ELi32ELi0EEESB_EES9_LST_1ELSV_1EiNS_10WmmaMatrixILSF_0ELS6_1ES9_SB_EELSX_1ESY_EENS10_INS11_IS1M_Li4ELm16EEEEEEENS1E_INS1F_INS0_29WmmaGemmSharedLoadTileBTraitsILS6_1ES9_S19_S1H_Li1152ENS7_ILi1ELi1ELi2ELi1EEENS7_ILi16ELi0ELi4608ELi1EEESB_EES9_LST_1ELSV_1EiNS1L_ILSF_1ELS6_1ES9_SB_EELSX_1ESY_EENS10_INS11_IS1V_Li2ELm16EEEEEEENS0_12GemmEpilogueINS0_28SimplifiedGemmEpilogueTraitsISC_NS0_13LinearScalingIfNS0_19FragmentMultiplyAddIffLb1EEEEEiNS0_28WmmaGemmEpilogueTraitsHelperISC_fS25_iEEEEEENS0_20IdentityBlockSwizzleEiNS0_17ClearAccumulatorsIfLi1EEEEEEEEEvNT_6ParamsE",
#     "compute_gemm__halfconst__halfconstfloatconstfloatfloatfloat" : "_Z12compute_gemmPK6__halfS1_PKfPfff"
# }


# Define the directories
directory1 = '/gpfs/alpine/scratch/brantran/gen150/accelwattch-artifact-appendix/accel-sim-framework/gpu-simulator/../hw_run/ppc64le_validation_traces/device-0/11.0'
directory2 = '/gpfs/alpine/scratch/brantran/gen150/accelwattch-artifact-appendix/accel-sim-framework/gpu-simulator/../hw_run/device-0/11.0'

# Define the output file
output_file = 'kernel_names.txt'

# Function to process the second directory
def process_kernel_names(nsight_dir_path,nvbit_trace_dir_path):
    # Loop through the application folders
    kernel_name_map = {}
    order_kernel_name_map = {}
    for app_folder in os.listdir(nsight_dir_path):
        order_kernel_name_map[app_folder] = []
        nsight_kernel_list = []
        app_path = os.path.join(nsight_dir_path, app_folder)
        # print(app_path)
        # Loop through the intermediate folders
        inter_dirs = os.listdir(app_path)
        if len(inter_dirs) != 1:
            print(app_folder,"does not only have one folder in it")
            continue
        inter_folder = inter_dirs[0]
        inter_path = os.path.join(app_path, inter_folder)
        # print(inter_path)
        # Search for the nsight file
        for file in os.listdir(inter_path):
            # print(file)
            if file.endswith('gpc__cycles_elapsed.0'):
                nsight_path = os.path.join(inter_path, file)
                # print(nsight_path)
                # Extract the kernel name from the nsight file
                with open(nsight_path, 'r') as file:
                    csv_started = False
                    reader = csv.reader(file)
                    # print(reader)
                    for row in reader:
                        # print(row)
                        if not csv_started:
                            if row == ['ID', 'Process ID', 'Process Name', 'Host Name', 'Kernel Name', 'Kernel Time', 'Context', 'Stream', 'Section Name', 'Metric Name', 'Metric Unit', 'Metric Value']:
                                csv_started = True
                            continue
                        else:
                            kernel = row[4].replace(",","")
                            kernel_plaintext = re.sub(r'\W+', '', kernel)
                            nsight_kernel_list.append(kernel_plaintext)
                # print(app_folder,nsight_kernel_list)
        #Once we have the nsight_kernel_list, now we need to compare to the kernel list
        nv_bit_path = os.path.join(nvbit_trace_dir_path,app_folder,inter_folder,"traces")
        traceg_files = [file for file in os.listdir(nv_bit_path) if file.startswith('kernel-') and file.endswith('.traceg')]
        # Check the number of kernel-*.traceg files
        if len(traceg_files) != len(nsight_kernel_list):
            print("Number of traceg files does not match the length of nsight_kernel_list.",app_folder)
            print(f"Number of traceg files: {len(traceg_files)}")
            print(f"Length of nsight_kernel_list: {len(nsight_kernel_list)}")
            # Continue with other operations or return if needed
            continue
        else:
            for i in range(1,len(nsight_kernel_list)+1):
                traceg_file = f"kernel-{i}.traceg"
                with open(os.path.join(nv_bit_path, traceg_file), 'r') as file:
                    first_line = file.readline().strip()
                nvbit_kernel_name = first_line.split()[3]
                nsight_kernel = nsight_kernel_list[i-1]
                # print(traceg_file,first_line,nsight_kernel)
                if nsight_kernel in kernel_name_map:
                    if kernel_name_map[nsight_kernel] != nvbit_kernel_name:
                        print(f"Conflicting kernel_name for nsight_kernel: {nvbit_kernel_name}")
                        print(f"Previous kernel_name file: {kernel_name_map[nsight_kernel]}")
                        print(f"Current traceg file: {traceg_file}")
                        continue  # Skip to the next kernel pair
                else:
                    kernel_name_map[nsight_kernel] =  nvbit_kernel_name

                    order_kernel_name_map[app_folder].append(nvbit_kernel_name)
    return kernel_name_map,order_kernel_name_map
kernel_name_map,kernel_numbering_map = process_kernel_names(directory2,directory1)


In [4]:
os.getcwd()

'/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework'

In [5]:
accel_sim_root = os.getcwd()

In [6]:
hw_stats_dir = os.path.join(accel_sim_root,"hw_run")
path = []
start_line = []
for dirpath, dirnames, filenames in os.walk(hw_stats_dir):
    for file in filenames:
        if "csv.nsight" in file:
            path.append(os.path.join(dirpath, file))
path

['/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/l1_bw_32f_20xiters/NO_ARGS/23.07.18-Tuesday--11:21:05.csv.nsight',
 '/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/parboil-mri-q/_i___data_large_input_64_64_64_dataset_bin__o_64_64_64_dataset_out/23.07.10-Monday--10:54:59.csv.nsight',
 '/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/histogram/NO_ARGS/23.07.10-Monday--10:54:59.csv.nsight',
 '/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/binomialOptions/NO_ARGS/23.07.10-Monday--10:54:59.csv.nsight',
 '/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/cutlass_perf_test_k1/__seed_2020___dist_0____m_2560___n_16___k_2560___kernels_wmma_gemm_nn____iterations_5___providers_cutlass/23.07.10-Monday--10:54:59.csv.

In [6]:
testing_string="""1	_Z12ComputeQ_GPUiiPfS_S_S_S_
2	_Z17histogram64KernelPjP5uint4j
3	_Z21binomialOptionsKernelv
4	_Z26quasirandomGeneratorKernelPfjj
5	_Z16inverseCNDKernelPfPjj
6	_Z15sobolGPU_kerneljjPjPf
7	_Z11mb_sad_calcPtS_ii
8	_Z14calculate_tempiPfS_S_iiiiffffff
9	findRangeK
10	findK
11	_Z4sradfiilPiS_S_S_PfS0_S0_S0_fS0_S0_
12	_Z11kmeansPointPfiiiPiS_S_S0_
13	_Z14CUDAkernel1DCTPfiiiy
14	_Z14CUDAkernel2DCTPfS_i
15	_Z15fwtBatch2KernelPfS_i
16	_Z15fwtBatch1KernelPfS_i
17	_Z22bpnn_layerforward_CUDAPfS_S_S_ii
18	_Z24bpnn_adjust_weights_cudaPfiS_iS_S_"""
obtained_list = [string.split('	')[1] for string in testing_string.split('\n')]
obtained_list
# for key,value in kernel_name_map.items():
#     if value not in obtained_list:
#         print(key)
#         print()

['_Z12ComputeQ_GPUiiPfS_S_S_S_',
 '_Z17histogram64KernelPjP5uint4j',
 '_Z21binomialOptionsKernelv',
 '_Z26quasirandomGeneratorKernelPfjj',
 '_Z16inverseCNDKernelPfPjj',
 '_Z15sobolGPU_kerneljjPjPf',
 '_Z11mb_sad_calcPtS_ii',
 '_Z14calculate_tempiPfS_S_iiiiffffff',
 'findRangeK',
 'findK',
 '_Z4sradfiilPiS_S_S_PfS0_S0_S0_fS0_S0_',
 '_Z11kmeansPointPfiiiPiS_S_S0_',
 '_Z14CUDAkernel1DCTPfiiiy',
 '_Z14CUDAkernel2DCTPfS_i',
 '_Z15fwtBatch2KernelPfS_i',
 '_Z15fwtBatch1KernelPfS_i',
 '_Z22bpnn_layerforward_CUDAPfS_S_S_ii',
 '_Z24bpnn_adjust_weights_cudaPfiS_iS_S_']

In [7]:
size = 41

total = np.zeros([1,size])
# for file in path:
#     print(file)
#     # skip first several lines of nsight output
#     with open(file) as myFile:
#         for num, line in enumerate(myFile, 1):
#             if "Disconnected" in line:
#                 start = num
#                 break
#     if start == 0:
#         print("output file error", file)
#         break
#     df = pd.read_csv(file,',',skiprows = start)

In [8]:
file = path[6]
file

'/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/quasirandomGenerator/NO_ARGS/23.07.10-Monday--10:54:59.csv.nsight'

In [9]:
for file in path:
    with open(file) as myFile:
        one_more=False
        for num, line in enumerate(myFile, 1):
            # print(num,line)
            if line.startswith('"ID",'):
                start = num
                break
    start

    offset=-1 #Two more lines of general info
    df = pd.read_csv(file,delimiter=',',skiprows = (start+offset))

    list(df.keys())

    app_names = df["Process Name"].to_numpy()[1:].astype(str)
    kernel_names = df["Kernel Name"].to_numpy()[1:].astype(str)
    sm__inst_executed = df["sm__inst_executed.sum"].to_numpy()[1:].astype(int)
    sm__sass_thread_inst_executed_op_fp16_pred_on = df["sm__sass_thread_inst_executed_op_fp16_pred_on.sum"].to_numpy()[1:].astype(int)
    sm__sass_thread_inst_executed_op_fp32_pred_on = df["sm__sass_thread_inst_executed_op_fp32_pred_on.sum"].to_numpy()[1:].astype(int)
    sm__sass_thread_inst_executed_op_fp64_pred_on = df["sm__sass_thread_inst_executed_op_fp64_pred_on.sum"].to_numpy()[1:].astype(int)
    sm__sass_thread_inst_executed_op_integer_pred_on = df["sm__sass_thread_inst_executed_op_integer_pred_on.sum"].to_numpy()[1:].astype(int)
    l1tex__t_sectors_pipe_lsu_mem_global_op_ld_lookup_hit = df["l1tex__t_sectors_pipe_lsu_mem_global_op_ld_lookup_hit.sum"].to_numpy()[1:].astype(int)
    sm__sass_thread_inst_executed_op_dfma_pred_on = df["sm__sass_thread_inst_executed_op_dfma_pred_on.sum"].to_numpy()[1:].astype(int)
    sm__sass_thread_inst_executed_op_dmul_pred_on = df["sm__sass_thread_inst_executed_op_dmul_pred_on.sum"].to_numpy()[1:].astype(int)
    l1tex__t_sectors_pipe_lsu_mem_global_op_ld_lookup_miss = df["l1tex__t_sectors_pipe_lsu_mem_global_op_ld_lookup_miss.sum"].to_numpy()[1:].astype(int)
    l1tex__t_sectors_pipe_lsu_mem_global_op_st_lookup_hit = df["l1tex__t_sectors_pipe_lsu_mem_global_op_st_lookup_hit.sum"].to_numpy()[1:].astype(int)
    l1tex__t_sectors_pipe_lsu_mem_global_op_st_lookup_miss = df["l1tex__t_sectors_pipe_lsu_mem_global_op_st_lookup_miss.sum"].to_numpy()[1:].astype(int)
    idc__requests = df["idc__requests.sum"].to_numpy()[1:].astype(int)
    idc__requests_hit = df["idc__requests_lookup_hit.sum"].to_numpy()[1:].astype(int)
    sm__sass_inst_executed_op_shared_ld = df["sm__sass_inst_executed_op_shared_ld.sum"].to_numpy()[1:].astype(int)
    sm__sass_inst_executed_op_shared_st = df["sm__sass_inst_executed_op_shared_st.sum"].to_numpy()[1:].astype(int)
    sm__inst_executed_pipe_alu = df["sm__inst_executed_pipe_alu.sum"].to_numpy()[1:].astype(int)
    sm__inst_executed_pipe_fma = df["sm__inst_executed_pipe_fma.sum"].to_numpy()[1:].astype(int)
    sm__sass_thread_inst_executed_ops_dadd_dmul_dfma_pred_on = df["sm__sass_thread_inst_executed_ops_dadd_dmul_dfma_pred_on.sum"].to_numpy()[1:].astype(int)
    sm__inst_executed_pipe_xu = df["sm__inst_executed_pipe_xu.sum"].to_numpy()[1:].astype(int)
    sm__inst_executed_pipe_tensor = df["sm__inst_executed_pipe_tensor.sum"].to_numpy()[1:].astype(int)
    sm__inst_executed_pipe_tex = df["sm__inst_executed_pipe_tex.sum"].to_numpy()[1:].astype(int)
    sm__inst_executed_pipe_fp16 = df["sm__inst_executed_pipe_fp16.sum"].to_numpy()[1:].astype(int)
    sm__inst_executed_pipe_fp64 = df["sm__inst_executed_pipe_fp64.sum"].to_numpy()[1:].astype(int)
    sm__pipe_alu_cycles_active = df["sm__pipe_alu_cycles_active.sum"].to_numpy()[1:].astype(int)
    sm__pipe_fma_cycles_active = df["sm__pipe_fma_cycles_active.sum"].to_numpy()[1:].astype(int)
    sm__pipe_fp64_cycles_active = df["sm__pipe_fp64_cycles_active.sum"].to_numpy()[1:].astype(int)
    sm__pipe_shared_cycles_active = df["sm__pipe_shared_cycles_active.sum"].to_numpy()[1:].astype(int)
    sm__pipe_tensor_cycles_active = df["sm__pipe_tensor_cycles_active.sum"].to_numpy()[1:].astype(int)
    lts__t_sectors_srcunit_tex_op_read_lookup_hit = df["lts__t_sectors_srcunit_tex_op_read_lookup_hit.sum"].to_numpy()[1:].astype(int)
    lts__t_sectors_srcunit_tex_op_read_lookup_miss = df["lts__t_sectors_srcunit_tex_op_read_lookup_miss.sum"].to_numpy()[1:].astype(int)
    lts__t_sectors_srcunit_tex_op_write_lookup_hit = df["lts__t_sectors_srcunit_tex_op_write_lookup_hit.sum"].to_numpy()[1:].astype(int)
    lts__t_sectors_srcunit_tex_op_write_lookup_miss = df["lts__t_sectors_srcunit_tex_op_write_lookup_miss.sum"].to_numpy()[1:].astype(int)
    lts__t_sectors_srcunit_tex_op_read = df["lts__t_sectors_srcunit_tex_op_read.sum"].to_numpy()[1:].astype(int)
    lts__t_sectors_srcunit_tex_op_write = df["lts__t_sectors_srcunit_tex_op_write.sum"].to_numpy()[1:].astype(int)
    dram__sectors_read = df["dram__sectors_read.sum"].to_numpy()[1:].astype(int)
    dram__sectors_write = df["dram__sectors_write.sum"].to_numpy()[1:].astype(int)
    sm__cycles_active = df["sm__cycles_active.avg"].to_numpy()[1:].astype(float)
    sm__cycles_active_total = df["sm__cycles_active.sum"].to_numpy()[1:].astype(float)
    sm__cycles_elapsed_avg = df["sm__cycles_elapsed.avg"].to_numpy()[1:].astype(float)
    sm__cycles_elapsed_total = df["sm__cycles_elapsed.sum"].to_numpy()[1:].astype(int)
    sm__warps_active = df["sm__warps_active.avg.pct_of_peak_sustained_active"].to_numpy()[1:].astype(float)
    active_threads = sm__warps_active * 32 / 100
    smsp__cycles_active = df["smsp__cycles_active.avg.pct_of_peak_sustained_elapsed"].to_numpy()[1:].astype(float)
    smsp__thread_inst_executed_per_inst_executed = df["smsp__thread_inst_executed_per_inst_executed.ratio"].to_numpy()[1:].astype(float)
    sm__sass_inst_executed_op_memory_128b = df["sm__sass_inst_executed_op_memory_128b.sum"].to_numpy()[1:].astype(float)
    sm__sass_inst_executed_op_memory_64b = df["sm__sass_inst_executed_op_memory_64b.sum"].to_numpy()[1:].astype(float)
    sm__sass_inst_executed_op_memory_32b = df["sm__sass_inst_executed_op_memory_32b.sum"].to_numpy()[1:].astype(float)
    sm__sass_inst_executed_op_memory_16b = df["sm__sass_inst_executed_op_memory_16b.sum"].to_numpy()[1:].astype(float)
    sm__sass_inst_executed_op_memory_8b = df["sm__sass_inst_executed_op_memory_8b.sum"].to_numpy()[1:].astype(float)
    sm__sass_thread_inst_executed = df["sm__sass_thread_inst_executed.sum"].to_numpy()[1:].astype(int)


    zeros = np.zeros(kernel_names.shape).astype(int)
    app_names,kernel_names

    kernel_output = []
    # kernel name, 0
    kernel_output.append(kernel_names)
    # insts, 1
    kernel_output.append(sm__inst_executed)
    # fp insts + int insts, 2
    fp_int_inst = sm__inst_executed - sm__sass_inst_executed_op_memory_128b - sm__sass_inst_executed_op_memory_64b - sm__sass_inst_executed_op_memory_32b - sm__sass_inst_executed_op_memory_16b - sm__sass_inst_executed_op_memory_8b
    kernel_output.append(fp_int_inst)
    # inst cache, 3
    kernel_output.append(zeros)
    # 4
    kernel_output.append(zeros)

    # L1 Read Hits, 5
    kernel_output.append(l1tex__t_sectors_pipe_lsu_mem_global_op_ld_lookup_hit)
    # L1 Read Miss, 6
    kernel_output.append(l1tex__t_sectors_pipe_lsu_mem_global_op_ld_lookup_miss)
    # L1 write Hit, 7
    kernel_output.append(l1tex__t_sectors_pipe_lsu_mem_global_op_st_lookup_hit)
    # L1 write Hit, 8
    kernel_output.append(l1tex__t_sectors_pipe_lsu_mem_global_op_st_lookup_miss)
    # constant cache, 9
    kernel_output.append(idc__requests)
    # shared memory, 10
    kernel_output.append(sm__sass_inst_executed_op_shared_ld + sm__sass_inst_executed_op_shared_st)
    # register, 11
    kernel_output.append(zeros)
    # 12
    kernel_output.append(zeros)
    # 13
    kernel_output.append(zeros)
    # total ALU, 14
    alu = sm__inst_executed_pipe_alu
    kernel_output.append(alu)
    # 15
    kernel_output.append(zeros)
    # 16
    kernel_output.append(zeros)
    # int and fp mul/mad, 17
    fma = sm__inst_executed_pipe_fma
    kernel_output.append(fma)
    # 18
    kernel_output.append(zeros)
    xu = sm__inst_executed_pipe_xu
    # 19
    kernel_output.append(xu)
    # 20
    kernel_output.append(zeros)
    # 21
    kernel_output.append(zeros)
    # 22
    kernel_output.append(zeros)
    # Double Precision Mul/Mad thread insts, 23
    kernel_output.append(sm__sass_thread_inst_executed_op_dfma_pred_on + sm__sass_thread_inst_executed_op_dmul_pred_on)
    # Tensor Core thread insts, 24
    tensor = sm__inst_executed_pipe_tensor
    kernel_output.append(tensor)
    # Texture Unit accesses, 25
    tex = sm__inst_executed_pipe_tex
    kernel_output.append(tex)
    # DRAM reads, 26
    kernel_output.append(dram__sectors_read)
    # DRAM writes, 27
    kernel_output.append(dram__sectors_write)
    # DRAM precharge, 28
    kernel_output.append(zeros)
    # L2 read Hits, 29
    kernel_output.append(lts__t_sectors_srcunit_tex_op_read_lookup_hit)
    # L2 read Miss, 30
    kernel_output.append(lts__t_sectors_srcunit_tex_op_read_lookup_miss)
    # L2 Write Hits, 31
    kernel_output.append(lts__t_sectors_srcunit_tex_op_write_lookup_hit)
    # L2 Write Miss, 32
    kernel_output.append(lts__t_sectors_srcunit_tex_op_write_lookup_miss)
    # Flits from SMs to memory partitions + Flits from memory partitions to SMs, 33
    flits = (lts__t_sectors_srcunit_tex_op_read + lts__t_sectors_srcunit_tex_op_write) * 2 
    kernel_output.append(flits)
    # Average SM pipeline duty cycle, 34
    duty_cycle = sm__pipe_alu_cycles_active + sm__pipe_fma_cycles_active + sm__pipe_fp64_cycles_active + sm__pipe_shared_cycles_active + sm__pipe_tensor_cycles_active
    duty_cycle = duty_cycle / sm__cycles_elapsed_total / 5
    kernel_output.append(duty_cycle)
    # Average Num Idle SMs, 35
    kernel_output.append((1 - smsp__cycles_active/100) * 80)
    # Average active threads per warp, 36
    kernel_output.append(smsp__thread_inst_executed_per_inst_executed)
    # idc__requests_lookup_hit.sum, 37
    kernel_output.append(idc__requests_hit)
    # total cycle, 38
    kernel_output.append(sm__cycles_elapsed_total/80)
    # 39
    kernel_output.append(sm__sass_thread_inst_executed)

    # create numpy array
    kernel_output = np.array(kernel_output)
    kernel_output = np.transpose(kernel_output)

    kernel_output

    # kernel_names

    unique_names = []
    accelsim_names = []
    output = []
    for kernel in kernel_names:
        kernel = kernel.replace(",","")
        kernel_plaintext = re.sub(r'\W+', '', kernel)
        # print(kernel_plaintext)
        if kernel not in unique_names and kernel_plaintext in kernel_name_map:
            # print(kernel_plaintext)
            unique_names.append(kernel)
            # accelsim_names.append(kernel_name_map.get(kernel_plaintext,""))
            accelsim_names.append(kernel_name_map.get(kernel_plaintext,kernel))
    # "voidmergeSortSharedKernel1uunsignedintunsignedintunsignedintunsignedintunsignedint"==
    # "voidmergeSortSharedKernelunsignedint1unsignedintunsignedintunsignedintunsignedintunsignedint"

    # print(unique_names)

    apps = []
    # create array for each unique kernel
    # same kernel are combined
    for name in unique_names:
        apps.append(app_names[0])
    apps = np.array(apps)
    apps = np.reshape(apps, [-1,1])
    output = np.array(accelsim_names)
    output = np.reshape(output, [-1,1])
    output = np.append(apps,output,axis = 1)
    tmp_output = np.zeros_like(kernel_output[0])
    tmp_output = np.reshape(tmp_output,[1,-1])
    tmp_output = tmp_output[1:,1:]


    # print("total kernel combined in last app",kernel_names.shape[0])
    for name in unique_names:
        # replace all symbols and spaces
        name_text = re.sub(r'\W+', '', name)
        combine_kernel = np.zeros_like(kernel_output[0])
        combine_kernel = np.reshape(combine_kernel,[1,-1])
        count = 0
        for kernel in range(0,kernel_names.shape[0]):
            # combine_kernel = np.append(combine_kernel,np.reshape(kernel_output[kernel],[1,-1]),axis = 0)
            # replace all symbols and spaces to compare alphabets only
            kernel_text = re.sub(r'\W+', '', kernel_names[kernel])
            if name_text in kernel_text:
            # if name.split("(")[0] in kernel_names[kernel] or \
            #     name.split("<")[0] in kernel_names[kernel]:
                combine_kernel = np.append(combine_kernel,np.reshape(kernel_output[kernel],[1,-1]),axis = 0)
                count = count + 1
        combine_kernel = combine_kernel[1:]
        # print("kernel", "<" + kernel_names[kernel] + ">","found",count,"times")

        if(combine_kernel.shape[0]) == 0:
            # parsing error or string comparision error
            exit("ERROR,", kernel_names[kernel], "not found")

        print("#num_kernels: ",count,name[:20])
        # calcualte elapsed cycle (total elapsed cycle / # of kernel launch)
        combine_kernel[:,38] = combine_kernel[:,38].astype(float) / float(count)
        # Num_Idle_SMs = Num_Idle_SMs / # of kernel launch
        # combine_kernel[:,35] = combine_kernel[:,35].astype(float) / float(count)
        # tmp = np.reshape(np.zeros_like(combine_kernel[0]),[1,-1])
        # calculate metrics that need to be added for the same kernel
        tmp_sum = combine_kernel[:,1:34].astype(float) / float(count)
        tmp_sum = tmp_sum.sum(axis=0)
        tmp_sum = np.reshape(tmp_sum, [1,-1])
        # calculate metrics that need to be averages for the same kernel
        tmp_avg = combine_kernel[:,34:37].astype(float)
        tmp_avg = tmp_avg.mean(axis=0)
        tmp_avg = np.reshape(tmp_avg, [1,-1])
        tmp = np.append(tmp_sum,tmp_avg,axis = 1)
        # sum again
        tmp_sum = combine_kernel[:,37:].astype(float)
        tmp_sum = tmp_sum.sum(axis=0)
        tmp_sum = np.reshape(tmp_sum, [1,-1])
        tmp = np.append(tmp,tmp_sum,axis = 1)
        tmp_output = np.append(tmp_output,tmp, axis = 0)
    # add kernel to the total array (for all apps)
    output = np.append(output,tmp_output,axis = 1)
    total = np.append(total,output,axis = 0)
    print(file)
    print()
# first line are all 0s when creating the array using np.zeros
# so remove first line
total = total[1:]
# un-comment these to save all metrics (used for verification)
# np.savetxt("output.csv",total,delimiter=',',fmt='%s')
# exit()

/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/l1_bw_32f_20xiters/NO_ARGS/23.07.18-Tuesday--11:21:05.csv.nsight

#num_kernels:  1 ComputePhiMag_GPU(fl
#num_kernels:  2 ComputeQ_GPU(int int
/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/parboil-mri-q/_i___data_large_input_64_64_64_dataset_bin__o_64_64_64_dataset_out/23.07.10-Monday--10:54:59.csv.nsight

#num_kernels:  17 histogram64Kernel(un
#num_kernels:  17 mergeHistogram64Kern
#num_kernels:  17 histogram256Kernel(u
#num_kernels:  17 mergeHistogram256Ker
/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/histogram/NO_ARGS/23.07.10-Monday--10:54:59.csv.nsight

#num_kernels:  1 binomialOptionsKerne
/gpfs/alpine/gen150/scratch/brantran/accelwattch-artifact-appendix/accel-sim-framework/hw_run/device-0/11.0/binomialOptions/NO_ARGS/23.07.10-Monday--10:54:59.csv.nsight

#num_kern

In [10]:
# with np.printoptions(threshold=np.inf):
#     print(total)

In [12]:

# select metrics used in Accel-Wattch
# these constants needs to be changed if new metrics are added
selected = np.empty((total.shape[0],0))
selected = np.append(selected,total[:,0:2],axis = 1)
selected = np.append(selected,total[:,6:12],axis = 1)
selected = np.append(selected,total[:,27:29],axis = 1)
selected = np.append(selected,total[:,30:37],axis = 1)
selected = np.append(selected,total[:,39:40],axis = 1)
# selected = np.append(selected,total[:,30:31],axis = 1)
# print(selected[0])

# add chip voltage - all 1 volts
ones = np.ones((selected.shape[0],1))
selected = np.concatenate((selected,ones),1)

#I want to sort things
# Define the custom sorting key function
def custom_sort_key(row):
    first_element = row[0]
    
    # Check the custom rules for sorting
    if first_element == 'Benchmark':
        return 0
    elif 'rodinia' in first_element:
        return 1
    elif 'parboil' in first_element:
        return 2
    elif 'cutlass' in first_element:
        return 5
    elif 'cudaTensor' in first_element:
        return 6
    else:
        return 3  # All other cases fall into group 4
order_index = []
for i, row in enumerate(selected):
    # print(i,str(custom_sort_key(row))+row[0],row,)
    order_index.append(str(custom_sort_key(row))+row[0]+f"|{i}")
order_index.sort()
order_index = [int(index.split('|')[1]) for index in order_index]
selected = selected[order_index]
selected

# add csv titles
csv_titles = ["Benchmark", "Kernel", "L1_RH","L1_RM","L1_WH","L1_WM","CC_ACC","SHRD_ACC","DRAM_Rd","DRAM_Wr","L2_RH","L2_RM","L2_WH","L2_WM","NOC","Pipeline_Duty","Num_Idle_SMs","Elapsed_Cycles","Chip Voltage"]
csv_titles = np.array(csv_titles)
csv_titles = np.reshape(csv_titles,(1,19))
selected = np.concatenate((csv_titles,selected),0)

# Sort the array using the sorted indices
# sorted_selected = selected[sorted_indices]
# print(selected)
# save output
np.savetxt("tesla_hw_perf.csv",selected,delimiter=',',fmt='%s')

<h1> Comparing Full Kernel List to hw_perf.csv </h1>

In [13]:
#I just want to know what kernels are in the original that don't appear in my gathered
import csv
hw_perf_kernel_list = []
with open("hw_perf.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        hw_perf_kernel_list.append((row[0],row[1]))
# print(hw_perf_kernel_list)

In [14]:
all_kernel_list = []
with open("tesla_hw_perf.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        all_kernel_list.append((row[0],row[1]))

for kernel in hw_perf_kernel_list:
    if kernel not in all_kernel_list:
        print(kernel)

('dct8x8', '_Z14CUDAkernel1DCTPfiiiy')


In [15]:
#What is dct8x8 even in context of the tesla all gather
# for (benchmark,kernel) in all_kernel_list:
#     if benchmark == 'dct8x8':
#         print(kernel)

In [26]:
# The difference is just that the original dct8x8 has an additional 'y' at tbe end: _Z14CUDAkernel1DCTPfiiiy

In [27]:
kernelnames = { 'backprop-rodinia-3.1' : '_Z22bpnn_layerforward_CUDAPfS_S_S_ii',
                    'binomialOptions' : '_Z21binomialOptionsKernelv',
                    'b+tree-rodinia-3.1' : 'findRangeK',
                    'dct8x8' : '_Z14CUDAkernel1DCTPfiiiy',
                    'fastWalshTransform' : '_Z15fwtBatch2KernelPfS_i',
                    'histogram' : '_Z17histogram64KernelPjP5uint4j',
                    'hotspot-rodinia-3.1' : '_Z14calculate_tempiPfS_S_iiiiffffff',
                    'kmeans-rodinia-3.1' : '_Z11kmeansPointPfiiiPiS_S_S0_',
                    'mergeSort' : '_Z30mergeElementaryIntervalsKernelILj1EEvPjS0_S0_S0_S0_S0_jj',
                    'pathfinder-rodinia-3.1' : '_Z14dynproc_kerneliPiS_S_iiii',
                    'quasirandomGenerator' : '_Z26quasirandomGeneratorKernelPfjj',
                    'sobolQRNG' : '_Z15sobolGPU_kerneljjPjPf',
                    'srad_v1-rodinia-3.1' : '_Z4sradfiilPiS_S_S_PfS0_S0_S0_fS0_S0_',
                    'parboil-mri-q' : '_Z12ComputeQ_GPUiiPfS_S_S_S_',	
                    'parboil-sad' : '_Z11mb_sad_calcPtS_ii',
                    'parboil-sgemm' : '_Z9mysgemmNTPKfiS0_iPfiiff',	
                    'cutlass_perf_test_k1' : '',
                    'cutlass_perf_test_k2' : '',
                    'cutlass_perf_test_k3' : '',
                    'cudaTensorCoreGemm' : '',
                    }
qv100_kernel_name_number={}
unique_kernel_names = []
for benchmark in kernelnames:
    if "cutlass" in benchmark:
        qv100_kernel_name_number[benchmark]=benchmark
        unique_kernel_names.append(benchmark)   
    if "dct8x8" in benchmark:
        qv100_kernel_name_number[benchmark]=kernelnames[benchmark].replace("iiiy","iii")
        unique_kernel_names.append(kernelnames[benchmark].replace("iiiy","iii"))           
    else:
        qv100_kernel_name_number[benchmark+"_k1"]=kernelnames[benchmark]
        unique_kernel_names.append(kernelnames[benchmark])
kernelnames = { 'backprop-rodinia-3.1' : '_Z24bpnn_adjust_weights_cudaPfiS_iS_S_',
        'binomialOptions' : '_Z21binomialOptionsKernelv',
        'b+tree-rodinia-3.1' : 'findK',
        'dct8x8' : '_Z14CUDAkernel2DCTPfS_i',
        'fastWalshTransform' : '_Z15fwtBatch1KernelPfS_i',
        'histogram' : '_Z18histogram256KernelPjS_j',
        'hotspot-rodinia-3.1' : '_Z14calculate_tempiPfS_S_iiiiffffff',
        'kmeans-rodinia-3.1' : '_Z11kmeansPointPfiiiPiS_S_S0_',
        'mergeSort' : '_Z21mergeSortSharedKernelILj1EEvPjS0_S0_S0_j',
        'pathfinder-rodinia-3.1' : '_Z14dynproc_kerneliPiS_S_iiii',
        'quasirandomGenerator' : '_Z16inverseCNDKernelPfPjj',
        'sobolQRNG' : '_Z15sobolGPU_kerneljjPjPf',
        'srad_v1-rodinia-3.1' : '_Z4sradfiilPiS_S_S_PfS0_S0_S0_fS0_S0_',	
        'parboil-mri-q' : '_Z12ComputeQ_GPUiiPfS_S_S_S_',	
        'parboil-sad' : '_Z11mb_sad_calcPtS_ii',
        'parboil-sgemm' : '_Z9mysgemmNTPKfiS0_iPfiiff',
        'cutlass_perf_test_k1' : '',
        'cutlass_perf_test_k2' : '',	
        'cutlass_perf_test_k3' : '',
        'cudaTensorCoreGemm' : '',
}
for benchmark in kernelnames:
    if kernelnames[benchmark] in unique_kernel_names:
        continue
    if len(kernelnames[benchmark]) > 0:
        qv100_kernel_name_number[benchmark+"_k2"]=kernelnames[benchmark]
        unique_kernel_names.append(kernelnames[benchmark])
qv100_kernel_name_number = {v:k for k,v in qv100_kernel_name_number.items()}
qv100_kernel_name_number

{'_Z22bpnn_layerforward_CUDAPfS_S_S_ii': 'backprop-rodinia-3.1_k1',
 '_Z21binomialOptionsKernelv': 'binomialOptions_k1',
 'findRangeK': 'b+tree-rodinia-3.1_k1',
 '_Z14CUDAkernel1DCTPfiii': 'dct8x8',
 '_Z15fwtBatch2KernelPfS_i': 'fastWalshTransform_k1',
 '_Z17histogram64KernelPjP5uint4j': 'histogram_k1',
 '_Z14calculate_tempiPfS_S_iiiiffffff': 'hotspot-rodinia-3.1_k1',
 '_Z11kmeansPointPfiiiPiS_S_S0_': 'kmeans-rodinia-3.1_k1',
 '_Z30mergeElementaryIntervalsKernelILj1EEvPjS0_S0_S0_S0_S0_jj': 'mergeSort_k1',
 '_Z14dynproc_kerneliPiS_S_iiii': 'pathfinder-rodinia-3.1_k1',
 '_Z26quasirandomGeneratorKernelPfjj': 'quasirandomGenerator_k1',
 '_Z15sobolGPU_kerneljjPjPf': 'sobolQRNG_k1',
 '_Z4sradfiilPiS_S_S_PfS0_S0_S0_fS0_S0_': 'srad_v1-rodinia-3.1_k1',
 '_Z12ComputeQ_GPUiiPfS_S_S_S_': 'parboil-mri-q_k1',
 '_Z11mb_sad_calcPtS_ii': 'parboil-sad_k1',
 '_Z9mysgemmNTPKfiS0_iPfiiff': 'parboil-sgemm_k1',
 'cutlass_perf_test_k1': 'cutlass_perf_test_k1',
 '': 'cudaTensorCoreGemm_k1',
 'cutlass_perf_test

In [28]:
tesla_kernel_name_number = {
 '_Z11mb_sad_calcPtS_ii\n': 'parboil-sad_k1',
 '_Z12compute_gemmPK6__halfS1_PKfPfff\n': 'cudaTensorCoreGemm_k1',
 '_Z14calculate_tempiPfS_S_iiiiffffff\n': 'hotspot-rodinia-3.1_k1',
 '_Z14modulateKernelPfS_i\n': 'fastWalshTransform_k3',
 '_Z15fwtBatch1KernelPfS_i\n': 'fastWalshTransform_k2',
 '_Z15fwtBatch2KernelPfS_i\n': 'fastWalshTransform_k1',
 '_Z15sobolGPU_kerneljjPjPf\n': 'sobolQRNG_k1',
 '_Z17histogram64KernelPjP5uint4j\n': 'histogram_k1',
 '_Z17larger_sad_calc_8Ptii\n': 'parboil-sad_k2',
 '_Z18larger_sad_calc_16Ptii\n': 'parboil-sad_k3',
 '_Z21mergeSortSharedKernelILj1EEvPjS0_S0_S0_j\n': 'mergeSort_k1',
 '_Z22bpnn_layerforward_CUDAPfS_S_S_ii\n': 'backprop-rodinia-3.1_k1',
 '_Z22mergeHistogram64KernelPjS_j\n': 'histogram_k2',
 '_Z24bpnn_adjust_weights_cudaPfiS_iS_S_\n': 'backprop-rodinia-3.1_k2',
 '_Z25generateSampleRanksKernelILj1EEvPjS0_S0_jjj\n': 'mergeSort_k2',
 '_Z26mergeRanksAndIndicesKernelPjS_jjj\n': 'mergeSort_k3',
 '_Z30mergeElementaryIntervalsKernelILj1EEvPjS0_S0_S0_S0_S0_jj\n': 'mergeSort_k4',
 '_Z4sradfiilPiS_S_S_PfS0_S0_S0_fS0_S0_\n': 'srad_v1-rodinia-3.1_k4',
 '_Z5srad2fiilPiS_S_S_PfS0_S0_S0_S0_S0_\n': 'srad_v1-rodinia-3.1_k5',
 '_Z6reduceliiPfS_\n': 'srad_v1-rodinia-3.1_k3',
 '_Z7extractlPf\n': 'srad_v1-rodinia-3.1_k1',
 '_Z7preparelPfS_S_\n': 'srad_v1-rodinia-3.1_k2',
 '_Z9mysgemmNTPKfiS0_iPfiiff\n': 'parboil-sgemm_k1',
 'findK\n': 'b+tree-rodinia-3.1_k2',
 'findRangeK\n': 'b+tree-rodinia-3.1_k1'
}
tesla_kernel_name_number = {k.strip():v for k,v in tesla_kernel_name_number.items()}
tesla_kernel_name_number

{'_Z11mb_sad_calcPtS_ii': 'parboil-sad_k1',
 '_Z12compute_gemmPK6__halfS1_PKfPfff': 'cudaTensorCoreGemm_k1',
 '_Z14calculate_tempiPfS_S_iiiiffffff': 'hotspot-rodinia-3.1_k1',
 '_Z14modulateKernelPfS_i': 'fastWalshTransform_k3',
 '_Z15fwtBatch1KernelPfS_i': 'fastWalshTransform_k2',
 '_Z15fwtBatch2KernelPfS_i': 'fastWalshTransform_k1',
 '_Z15sobolGPU_kerneljjPjPf': 'sobolQRNG_k1',
 '_Z17histogram64KernelPjP5uint4j': 'histogram_k1',
 '_Z17larger_sad_calc_8Ptii': 'parboil-sad_k2',
 '_Z18larger_sad_calc_16Ptii': 'parboil-sad_k3',
 '_Z21mergeSortSharedKernelILj1EEvPjS0_S0_S0_j': 'mergeSort_k1',
 '_Z22bpnn_layerforward_CUDAPfS_S_S_ii': 'backprop-rodinia-3.1_k1',
 '_Z22mergeHistogram64KernelPjS_j': 'histogram_k2',
 '_Z24bpnn_adjust_weights_cudaPfiS_iS_S_': 'backprop-rodinia-3.1_k2',
 '_Z25generateSampleRanksKernelILj1EEvPjS0_S0_jjj': 'mergeSort_k2',
 '_Z26mergeRanksAndIndicesKernelPjS_jjj': 'mergeSort_k3',
 '_Z30mergeElementaryIntervalsKernelILj1EEvPjS0_S0_S0_S0_S0_jj': 'mergeSort_k4',
 '_Z4sr

In [31]:
for kernel in qv100_kernel_name_number:
    if kernel not in tesla_kernel_name_number:
        # print(f"WARNING: {kernel} is in hw_perf.csv but not found in tesla (most likely due to errors in getting simulation")
        continue
    elif qv100_kernel_name_number[kernel] == tesla_kernel_name_number[kernel]:
        #print("Mapping between two is the same")
        continue
    else:
        print(f"'{tesla_kernel_name_number[kernel]}': '{qv100_kernel_name_number[kernel]}',")

'mergeSort_k4': 'mergeSort_k1',
'srad_v1-rodinia-3.1_k4': 'srad_v1-rodinia-3.1_k1',
'mergeSort_k1': 'mergeSort_k2',


In [30]:
tesla_kernel_name_to_hw_perf_name = {
'mergeSort_k4': 'mergeSort_k1',
'srad_v1-rodinia-3.1_k4': 'srad_v1-rodinia-3.1_k1',
'mergeSort_k1': 'mergeSort_k2',
}